In [32]:

import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib
import matplotlib.pyplot as plt

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 500)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.227574219e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 10000


In [63]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("../../../data/gps.dada")
print(dada.get_header())
dada = dada.combined_data()



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [64]:
array.steering_vector([0,0], wavelength)

array([ 0.29380879-0.95586421j, -0.38789908+0.92170185j,
       -0.80562962-0.59241955j,  0.89094245+0.45411623j,
        0.63052321+0.77617039j, -0.54047739-0.84135854j,
       -0.7179779 +0.69606589j,  0.99995061-0.00993887j,
       -0.99152701+0.12990066j, -0.91580951+0.40161293j,
        0.22433012+0.97451321j, -0.95112676+0.30880072j,
        0.38387379-0.92338557j,  0.70552644+0.7086836j ,
        0.77828376-0.62791273j, -0.49022161+0.87159783j,
        0.9929999 -0.1181152j ,  0.07410256-0.99725063j,
       -0.93833154-0.34573679j,  0.08121442-0.99669665j,
        0.67891664-0.73421536j,  0.94433605+0.3289824j ,
        0.53076907+0.84751649j,  0.01367786+0.99990645j,
        0.76778561+0.64070684j, -0.99951007+0.03129872j,
       -0.10965214-0.99397002j, -0.98281239+0.18460718j,
        0.31836857+0.94796701j, -0.86498433+0.50179887j,
        0.5036235 -0.86392324j, -0.07489689+0.99719128j,
        0.66157129-0.74988227j, -0.14029655-0.99010953j])

In [65]:
dada.shape

(208896, 57, 64, 2)

In [66]:
dada = dada[0:100000, core_indices, GPS_CHANNEL, POL].T
X = dada

In [67]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np
import astropy.units as u

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

lon_str = "21d26m38s"   # degrees, minutes, seconds format
lat_str = "-30d42m39.8s"  # negative for south

# Altitude (optional, in meters)
height = 1086.6 * u.m

# Create location
location = EarthLocation.from_geodetic(lon=lon_str, lat=lat_str, height=height)

print(location)

(5109360.133321233, 2006852.5860429057, -3238948.127478875) m


/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_54984/3372582233.py:16: DeprecationWarning:

datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



In [68]:


# Observer location (MeerKAT)
obs_location = EarthLocation(lat=lat*u.deg,
                         lon=lon*u.deg,
                         height=elevation_m*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print("Source RA/DEC")
print(f"Altitude: {altaz.alt:.2f}")
print(f"Azimuth: {altaz.az:.2f}")

theta_source = 90 - altaz.alt.degree
phi_source = (450 - altaz.az.degree) % 360

Source RA/DEC
Altitude: 24.03 deg
Azimuth: 229.33 deg


In [69]:
array_factor = array.steering_vector([np.deg2rad(phi_source), np.deg2rad(theta_source)], wavelength)
zero_array_factor = array.steering_vector([np.deg2rad(0), np.deg2rad(0)], wavelength)
#X = X.T
X_adj = X.copy()
for i, shift in enumerate(array_factor):
    X_adj[i, :] = X_adj[i, :] * array_factor[i].conj()

R = X @ X.conj().T / (N-1)
R_adj = X_adj @ X_adj.conj().T / (N-1)

In [70]:
import numpy as np

evals, evecs = np.linalg.eigh(R)
R

array([[2856289.5       +0.j   , -107345.03 -369030.78j ,
         236568.22 +222448.34j , ...,  364314.4  +192941.8j  ,
        -230099.58 -406805.8j  ,  -51150.094-309644.62j ],
       [-107345.03 +369030.78j , 2716665.8       +0.j   ,
        -182191.4   +34157.484j, ..., -218618.83 +231839.j   ,
         357499.7  -183257.83j ,   29180.729+426759.4j  ],
       [ 236568.22 -222448.34j , -182191.4   -34157.484j,
        2952032.5       +0.j   , ...,  628310.2  -155755.94j ,
        -324439.78 +510098.44j , -225723.95 -429334.34j ],
       ...,
       [ 364314.4  -192941.8j  , -218618.83 -231839.j   ,
         628310.2  +155755.94j , ..., 2910284.2       +0.j   ,
        -394563.16 +230827.47j , -169650.03 -446423.1j  ],
       [-230099.58 +406805.8j  ,  357499.7  +183257.83j ,
        -324439.78 -510098.3j  , ..., -394563.16 -230827.47j ,
        3040386.        +0.j   , -296668.44 +197962.55j ],
       [ -51150.094+309644.62j ,   29180.729-426759.4j  ,
        -225723.95 +429334.34j

In [71]:
R_adj

array([[2859155.8  -3.6707435e-02j,  259254.1  +2.8371500e+05j,
         280629.53 +1.6338047e+05j, ...,  255121.34 +3.2382444e+05j,
         461831.7  +7.1751695e+04j, -254389.22 +1.8379952e+05j],
       [ 259254.1  -2.8370816e+05j, 2719386.8  -2.0157753e-02j,
         185185.08 +8.2572529e+03j, ...,  318101.8  +1.8806520e+04j,
         243929.55 -3.1919875e+05j, -426619.1  +3.1164295e+04j],
       [ 280629.53 -1.6338003e+05j,  185185.08 -8.2573457e+03j,
        2954979.2  -3.9310660e-02j, ...,  596003.6  +2.5262045e+05j,
         -65154.016-6.0101288e+05j, -365122.56 +3.1931925e+05j],
       ...,
       [ 255121.34 -3.2383025e+05j,  318101.8  -1.8806611e+04j,
         596003.6  -2.5261691e+05j, ..., 2913210.2  +2.0677034e-02j,
        -124972.49 -4.3970809e+05j, -154388.84 +4.5192794e+05j],
       [ 461832.97 -7.1751430e+04j,  243928.22 +3.1919919e+05j,
         -65153.926+6.0100719e+05j, ..., -124971.05 +4.3971034e+05j,
        3040385.5  +5.9185608e-04j,  -52765.35 -3.5272816e+05j]

In [74]:
evals

array([ 1960220. ,  1976870. ,  1986177.6,  1992006.2,  2004538. ,
        2015902. ,  2028254. ,  2040608.9,  2051130.9,  2056981.9,
        2063606.6,  2073830.4,  2085582.2,  2089932.1,  2102652.2,
        2113521. ,  2123121.8,  2133641.2,  2141150. ,  2154055.2,
        2172299.2,  2186111.8,  2211036.2,  2216022. ,  2251667. ,
        2301144. ,  2358684.5,  2441199.5,  2501012.8,  2739985. ,
        3426652.5,  5909316.5,  7927220.5, 16605051. ], dtype=float32)

In [23]:
import matplotlib.pyplot as plt
plt.plot(np.abs(X[0, :]))
plt.show()

In [75]:


theta_steps = 750#6_000
phi_steps = 750#24_000

output = direction_of_arrival.MUSICDOA2D().get_directions(
    array,
    R_adj, 
    num_interferers=5,
    wavelength = wavelength,
    theta_min_deg = 0,
    theta_max_deg = 86,
    theta_steps=theta_steps,
    phi_min_deg = 0,
    phi_max_deg = 360,
    phi_steps=phi_steps,
)
output.head(10)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 562500/562500 [01:06<00:00, 8404.89it/s]


phi,theta,Q,Q_db
f64,f64,f64,f64
0.0,0.0,0.036262,-2.30355
0.0,0.11482,0.036051,-2.328999
0.0,0.22964,0.035921,-2.344624
0.0,0.344459,0.03249,-2.780642
0.0,0.459279,0.030218,-3.095518
0.0,0.574099,0.03802,-2.097986
0.0,0.688919,0.039161,-1.969581
0.0,0.803738,0.036558,-2.268265
0.0,0.918558,0.032006,-2.845847


In [59]:
len(output)

231736

In [60]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q_db'].quantile(0.999975)
high_Q = output.filter(pl.col('Q_db') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q_db',      # MUSIC power
    color='Q_db',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

The MJD I want is 60356.47317236924195




In [63]:
import util
importlib.reload(util)

tles_check = []
with open('tles_sgp4.tle', 'r') as f:
    while True:
        line1 = f.readline()
        if not line1 or line1.strip() == "":
            break  # stop if first line is empty

        line2 = f.readline()
        if not line2 or line2.strip() == "":
            break  # stop if second line is empty
        
        tles_check.append(util.TLE(line1.strip(), line2.strip()))

sat_check = []
for tle in tles_check:
    sat_check.append(util.get_position_at_time(tle, obstime, location))

sat_check

[(<Latitude 49.9011741 deg>, <Longitude 140.06565009 deg>),
 (<Latitude 16.64217106 deg>, <Longitude 91.63357231 deg>),
 (<Latitude -14.3998069 deg>, <Longitude 89.75286887 deg>),
 (<Latitude -15.55078374 deg>, <Longitude 226.63380823 deg>),
 (<Latitude -40.42825261 deg>, <Longitude 48.57907833 deg>),
 (<Latitude -76.46595843 deg>, <Longitude 213.61790564 deg>),
 (<Latitude -51.08696114 deg>, <Longitude 72.08651475 deg>),
 (<Latitude -53.13859519 deg>, <Longitude 285.93121387 deg>),
 (<Latitude -9.69607389 deg>, <Longitude 2.15128416 deg>),
 (<Latitude 76.9560202 deg>, <Longitude 197.63728034 deg>),
 (<Latitude 37.98097699 deg>, <Longitude 309.60795381 deg>),
 (<Latitude -17.82041055 deg>, <Longitude 159.98671405 deg>),
 (<Latitude -11.11461928 deg>, <Longitude 333.78781491 deg>),
 (<Latitude 36.85778348 deg>, <Longitude 20.40300613 deg>),
 (<Latitude -10.52907699 deg>, <Longitude 203.35030503 deg>),
 (<Latitude 4.09678194 deg>, <Longitude 141.1021063 deg>),
 (<Latitude -21.36389631 de

In [64]:
df

name,alt,az
str,f64,f64
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598
"""NAVSTAR 60 (USA 196)""",76.956355,197.642354
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433


In [59]:
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json
ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')

Loaded 54 satellites


In [60]:
from skyfield.api import wgs84, Topos

meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site = meerkat
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    # any of them should give essentially the same result.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt.degrees
    event_dict['az'] = az.degrees
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
print(len(sats_visible))
df.with_columns(theta=pl.lit(90) - pl.col('alt'), phi=(pl.lit(450) - pl.col('az'))%360).sort(by="alt", descending=False)


10


name,alt,az,theta,phi
str,f64,f64,f64,f64
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,85.903507,308.897567
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,81.951034,44.98524
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,73.357624,358.366296
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,67.443908,204.545606
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,53.142185,69.596966
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,52.019064,140.39203
"""NAVSTAR 64 (USA 206)""",44.369909,123.403165,45.630091,326.596835
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,44.627335,230.619031
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,40.096014,309.934402


In [61]:
output.with_columns(dist_from_theta=90 - pl.col('theta'), dist_from_phi = (450-pl.col('phi')) % 360).sort(by="Q", descending=True).head(10)

phi,theta,Q,Q_db,dist_from_theta,dist_from_phi
f64,f64,f64,f64,f64,f64
242.723632,30.771696,0.063721,0.0,59.228304,207.276368
191.775701,2.411215,0.062699,-0.070223,87.588785,258.224299
49.025367,25.489987,0.062034,-0.116556,64.510013,40.974633
152.363151,81.981308,0.061689,-0.140766,8.018692,297.636849
43.738318,45.009346,0.061405,-0.160761,44.990654,46.261682
73.05741,77.618158,0.061291,-0.168833,12.381842,16.94259
309.53271,63.839786,0.061161,-0.178068,26.160214,140.46729
291.748999,82.899866,0.060891,-0.197281,7.100134,158.251001
106.70227,61.543391,0.06077,-0.205928,28.456609,343.29773


In [50]:
evals_adj, evecs_adj = np.linalg.eigh(R_adj)
evals, evecs = np.linalg.eigh(R)
num_int = 5

sat_factor = array.steering_vector([np.deg2rad(230.619031), np.deg2rad(44.627335)], wavelength)


print(1/np.abs(sat_factor.conj().T @ evecs[:, :-num_int] @ evecs[:,:-num_int].conj().T @ sat_factor))
print(1/np.abs(sat_factor.conj().T @ evecs_adj[:, :-num_int] @ evecs_adj[:,:-num_int].conj().T @ sat_factor))

sat_factor = array.steering_vector([np.deg2rad(171.59215), np.deg2rad(82.043341)], wavelength)
print(1/np.abs(sat_factor.conj().T @ evecs_adj[:, :-num_int] @ evecs_adj[:,:-num_int].conj().T @ sat_factor))

0.034563616297615875
0.032586414348690314
0.03196086621724517


In [102]:
import scipy.optimize as opt
num_int = 7
def _opt(args):
    phi, theta = args
    phi *= 360
    theta *= 90

    sat_factor = array.steering_vector([np.deg2rad(171.59215), np.deg2rad(82.043341)], wavelength)
    array_factor = array.steering_vector([np.deg2rad(phi), np.deg2rad(theta)], wavelength)
    X_adj = X.copy()
    for i, shift in enumerate(array_factor):
        X_adj[i, :] = X_adj[i, :] * array_factor[i].conj()

    R_adj = X_adj @ X_adj.conj().T / (N-1)

    evals, evecs = np.linalg.eigh(R_adj)

    return np.abs(sat_factor.conj().T @ evecs[:, :-num_int] @ evecs[:,:-num_int].conj().T @ sat_factor)


result = opt.minimize(_opt, [0.3, 0.3], method="Powell")
print(result)
print(result.x[0] * 360 % 360)
print(result.x[1] * 90)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 19.148624363182986
       x: [ 4.405e+00 -3.181e-01]
     nit: 3
   direc: [[ 1.000e+00  0.000e+00]
           [ 2.010e-05 -8.927e-10]]
    nfev: 156
145.9555646020226
-28.62705577383894


In [32]:
sat_factor = array.steering_vector([np.deg2rad(0), np.deg2rad(0)], wavelength)


1 / np.abs(sat_factor.conj().T @ evecs[:, :-5] @ evecs[:,:-5].conj().T @ sat_factor)

np.float64(0.03650969600519887)

In [163]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


In [164]:
## Near-field limit calculation
1500**2 / (constants.c / 1.276e9)

9576625.173138944